In [50]:
%load_ext autoreload
%autoreload 2

In [60]:
from utils import (
    create_fasta_file, create_random_strand, reverse_complement,
    read_synthesized_strands_from_file, get_fastq_records, get_badread_strand_id
)
from checksum import CheckSum4
import random
from clustering import Clustering
import numpy as np
from strand_reconstruction import align
import sys
np.set_printoptions(threshold=sys.maxsize)

In [2]:
strand_length = 200
n_strands = 1004

strands = [create_random_strand(strand_length) for i in range(n_strands)]

In [80]:
checksum = CheckSum4(reference_length=200)

In [4]:
encoded_strands = checksum.encode(strands)

In [13]:
# Read cat.fasta - add encoding
# write fasta
# badread
# do clustering get candidates
# decode without apriori

In [5]:
original_strands, original_strand_ids = read_synthesized_strands_from_file(file_path=r"C:\Users\Parv\Doc\RA\Projects\incomplete_cycles\v2\raptor_data\cat.jpg_RU10 2.fasta")

In [6]:
encoded_strands = checksum.encode(original_strands)

In [7]:
create_fasta_file(ids=original_strand_ids, strands=encoded_strands, output_filepath='cat_encoded.fasta')

File saved as cat_encoded.fasta


In [8]:
# Raptor
records = get_fastq_records(fastq_filepath=r"C:\Users\Parv\Doc\RA\Projects\clustering_dna_storage\cat_encoded_synth.fasta\cat_encoded_synth.fasta")

0it [00:00, ?it/s]

22989it [00:00, 23077.36it/s]


In [9]:
reads = [str(i.seq) for i in records]
ids = [get_badread_strand_id(i) for i in records]

In [22]:
clustering = Clustering(strand_pool=reads, reference_length=204, original_strands=encoded_strands, strand_pool_ids=ids, original_strand_ids=original_strand_ids, distance_threshold=40)

In [94]:
candidates = clustering.generate_candidates(n_candidates=len(clustering.clustered_seqs))

100%|██████████| 1245/1245 [02:45<00:00,  7.50it/s]


Fixing candidate orientations
0.5140562248995983 candidates are reversed


In [ ]:
# We want to recover the original strands after checksum
decoded_strands = checksum.decode(candidates=candidates, n_reference_strands=len(original_strands), clustered_seqs=clustering.clustered_seqs, n_guesses=15, guesses=True)

clustering.original_strands = original_strands
clustering.fsm(candidates=decoded_strands)

100%|██████████| 1245/1245 [06:21<00:00,  3.26it/s]


0.16989247311827957 were reversed
930 Valid checksum strands found


AttributeError: 'Clustering' object has no attribute 'n'

In [92]:
import uuid
ids = [str(uuid.uuid4()) for i in range(len(decoded_strands))]
create_fasta_file(ids, decoded_strands, output_filepath='decoded_raptor.fasta')

File saved as decoded_raptor.fasta
